# Train U-Net for cervix/os detection on Colfax cluster

In [1]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
from datetime import datetime
    
# Project
project_common_path = os.path.abspath(os.path.join('..', 'common'))
if not project_common_path in sys.path:
    sys.path.append(project_common_path)    
    
    
from qsub_utils import submit_job
from qsub_utils import setup_configuration
from qsub_utils import PBS_CONFIGURATION    

In [3]:
from data_utils import RESOURCES_PATH, GENERATED_DATA, get_annotations

## Transform `sloth` annotations to label images

In [4]:
setup_configuration(nodes='1:knl7210:ram96gb')

In [7]:
sloth_annotations_filename = os.path.join(RESOURCES_PATH, 'cervix_os.json')
annotations = get_annotations(sloth_annotations_filename)
print(len(annotations))

!ls {GENERATED_DATA}/
ret = !ls {GENERATED_DATA}/trainval_labels | wc -l
print(ret[0])

if ret[0] != len(annotations):
    cmd = [
        "python",
        os.path.abspath(os.path.join("..", "scripts", "data_preparation_os_cervix_detection.py"))
    ]

    process, job_info = submit_job(cmd, 
                                   name='data_preparation_os_cervix_detection', 
                                   cwd=RESOURCES_PATH)

    try:
        while True:
            out = process.stdout.readline()    
            if len(out) > 0:        
                print out

            if process.poll() is not None and len(out) == 0:
                break
    except KeyboardInterrupt:
        !qdel {job_info['id']}
        time.sleep(1.0)
        !qstat

qsub: job 4767.c001 ready





  ########################################################################

  # Colfax Cluster - https://colfaxresearch.com/

  #      Date:           Thu Mar 30 01:01:48 PDT 2017

  #    Job ID:           4767.c001

  #      User:           u2459

  # Resources:           neednodes=1:knl7210:ram96gb,nodes=1:knl7210:ram96gb,walltime=24:00:00

  ########################################################################

  

Using Theano backend.

WARNING (theano.gof.cmodule): WARNING: your Theano flags `gcc.cxxflags` specify an `-march=X` flags.

         It is better to let Theano/g++ find it automatically, but we don't do it now

Keras version:  1.2.2

mkl_available:  True

('Found ', 639, ' annotations')

('--', u'0', u'Type_1', 0, '/', 639)

('--', u'7', u'Type_1', 1, '/', 639)

('--', u'10', u'Type_1', 2, '/', 639)

('--', u'12', u'Type_1', 3, '/', 639)

('--', u'13', u'Type_1', 4, '/', 639)

('--', u'14', u'Type_1', 5, '/', 639)

('--', u'27', u'Type_1

In [10]:
!qstat

In [12]:
sloth_annotations_filename = os.path.join(RESOURCES_PATH, 'cervix_os.json')
annotations = get_annotations(sloth_annotations_filename)
print(len(annotations))

!ls {GENERATED_DATA}/
ret = !ls {GENERATED_DATA}/trainval_labels | wc -l
print(ret[0])

assert len(annotations) == int(ret[0]), "WTF"

639
data_preparation_os_cervix_detection.launch  pred_labels
labels_gray				     trainval_labels
os_cervix_stats.npz
639


Check data iterator

In [ ]:
import time 

cmd = [
    "python",
     os.path.abspath(os.path.join("..", "scripts", "test_image_mask_generator.py"))
]

process, job_info = submit_job(cmd, 
                               name='test_image_mask_generator', 
                               cwd=RESOURCES_PATH)

try:
    while True:
        out = process.stdout.readline()    
        if len(out) > 0:        
            print out

        if process.poll() is not None and len(out) == 0:
            break
except KeyboardInterrupt:
    !qdel {job_info['id']}
    time.sleep(1.0)
    !qstat

## Train the network

In [5]:
setup_configuration(nodes='1:knl7210:ram96gb')

In [1]:
import time 

cmd = [
    "python",
    os.path.abspath(os.path.join("..", "scripts", "unet_cervix_os_detection_with_keras.py"))
]

process, job_info = submit_job(cmd, 
                               name='unet_cervix_os_detection_with_keras', 
                               cwd=RESOURCES_PATH)

try:
    while True:
        out = process.stdout.readline()    
        if len(out) > 0:        
            print out

        if process.poll() is not None and len(out) == 0:
            break
except KeyboardInterrupt:
    !qdel {job_info['id']}
    time.sleep(1.0)
    !qstat

NameError: name 'os' is not defined

In [8]:
!qstat